<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Merging" data-toc-modified-id="Merging-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merging</a></span><ul class="toc-item"><li><span><a href="#SV-artifact" data-toc-modified-id="SV-artifact-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>SV artifact</a></span></li><li><span><a href="#rep-seqs" data-toc-modified-id="rep-seqs-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>rep-seqs</a></span></li><li><span><a href="#Taxonomy" data-toc-modified-id="Taxonomy-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Taxonomy</a></span></li></ul></li><li><span><a href="#Alignment" data-toc-modified-id="Alignment-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Alignment</a></span><ul class="toc-item"><li><span><a href="#Creating-alignment" data-toc-modified-id="Creating-alignment-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Creating alignment</a></span></li><li><span><a href="#Masking-alignment" data-toc-modified-id="Masking-alignment-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Masking alignment</a></span></li></ul></li><li><span><a href="#Phylogeny" data-toc-modified-id="Phylogeny-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Phylogeny</a></span><ul class="toc-item"><li><span><a href="#Unrooted-tree" data-toc-modified-id="Unrooted-tree-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Unrooted tree</a></span></li><li><span><a href="#Rooted-tree" data-toc-modified-id="Rooted-tree-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Rooted tree</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Merge results from all per-MiSeq-run `LLA` jobs
* Merging feature tables for multiple sequencing runs:
   * MiSeq-Run0116
   * MiSeq-Run0122
   * MiSeq-Run0126
   * **NOT** MiSeq-Run187 (failed run)
   * MiSeq-run0189
* Then running standard processing:
  * dataset summary
  * taxonomy
  * phylogeny

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/'

run_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/'
miseq_runs = c('Run0116', 'Run0122', 'Run0126', 'Run0189', 'Run0190')

# params
conda_env = 'qiime2-2019.10'
threads = 24

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
df.dims()
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/ 


# Merging

## SV artifact

In [4]:
# artifacts for individual runs
P = file.path(run_dir, '{run}', 'table_merged_filt.qza')
runs = miseq_runs %>% as.list %>%
    lapply(function(x) glue::glue(P, run=x))
        
runs

[[1]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/table_merged_filt.qza

[[2]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/table_merged_filt.qza

[[3]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/table_merged_filt.qza

[[4]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/table_merged_filt.qza

[[5]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/table_merged_filt.qza

In [5]:
# function to merge tables
merge_tables = function(in_tbls, out_tbl, conda_env){
    cmd = 'qiime feature-table merge --i-tables {in_tbls} --o-merged-table {out_tbl} --p-overlap-method sum'
    cmd = glue::glue(cmd, in_tbls=in_tbls, out_tbl=out_tbl)
    cat('CMD:', cmd, '\n')
    ret = bash_job(cmd, conda_env=conda_env, stderr=TRUE)
    cat(ret, '\n')
    return(out_tbl)
}

In [6]:
# merging
table_merged_file = file.path(work_dir, 'table_merged_filt.qza')
table_merged_file = merge_tables(paste(runs, collapse=' '), table_merged_file, conda_env)
cat('Output file:', table_merged_file, '\n')

CMD: qiime feature-table merge --i-tables /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/table_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/table_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/table_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/table_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/table_merged_filt.qza --o-merged-table /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//table_merged_filt.qza --p-overlap-method sum 
Saved FeatureTable[Frequency] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//table_merged_filt.qza 
Output file: /ebio/abt3_projects/Georg_animal_feces

## rep-seqs

In [7]:
# artifacts for individual runs
P = file.path(run_dir, '{run}', 'rep-seqs_merged_filt.qza')
runs = miseq_runs %>% as.list %>%
    lapply(function(x) glue::glue(P, run=x))
        
runs

[[1]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/rep-seqs_merged_filt.qza

[[2]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/rep-seqs_merged_filt.qza

[[3]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/rep-seqs_merged_filt.qza

[[4]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/rep-seqs_merged_filt.qza

[[5]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/rep-seqs_merged_filt.qza

In [8]:
# function to merge seqs 
merge_seqs = function(in_seqs, out_seq, conda_env){
    cmd = 'qiime feature-table merge-seqs --i-data {in_seqs} --o-merged-data {out_seq}'
    cmd = glue::glue(cmd, in_seqs=in_seqs, out_tbl=out_seq)
    cat('CMD:', cmd, '\n')
    ret = bash_job(cmd, conda_env=conda_env, stderr=TRUE)
    cat(ret, '\n')
    return(out_seq)
}

In [9]:
# merging
seqs_merged_file = file.path(work_dir, 'rep-seqs_merged_filt.qza')
seqs_merged_file = merge_seqs(paste(runs, collapse=' '), seqs_merged_file, conda_env)
cat('Output file:', seqs_merged_file, '\n')

CMD: qiime feature-table merge-seqs --i-data /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/rep-seqs_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/rep-seqs_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/rep-seqs_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/rep-seqs_merged_filt.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/rep-seqs_merged_filt.qza --o-merged-data /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//rep-seqs_merged_filt.qza 
Saved FeatureData[Sequence] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//rep-seqs_merged_filt.qza 
Output file: /ebio/abt3_projects/Georg_animal_feces/d

## Taxonomy

In [10]:
# artifacts for individual runs
P = file.path(run_dir, '{run}', 'taxonomy.qza')
runs = miseq_runs %>% as.list %>%
    lapply(function(x) glue::glue(P, run=x))
        
runs

[[1]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/taxonomy.qza

[[2]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/taxonomy.qza

[[3]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/taxonomy.qza

[[4]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/taxonomy.qza

[[5]]
/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/taxonomy.qza

In [11]:
# function to merge tax 
merge_tax = function(in_taxs, out_tax, conda_env){
    cmd = 'qiime feature-table merge-taxa --i-data {in_seqs} --o-merged-data {out_tax}'
    cmd = glue::glue(cmd, in_seqs=in_taxs, out_tbl=out_tax)
    cat('CMD:', cmd, '\n')
    ret = bash_job(cmd, conda_env=conda_env, stderr=TRUE)
    cat(ret, '\n')
    return(out_tax)
}

In [12]:
# merging
tax_merged_file = file.path(work_dir, 'taxonomy.qza')
tax_merged_file = merge_tax(paste(runs, collapse=' '), tax_merged_file, conda_env)
cat('Output file:', tax_merged_file, '\n')

CMD: qiime feature-table merge-taxa --i-data /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0116/taxonomy.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0122/taxonomy.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0126/taxonomy.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0189/taxonomy.qza /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA//Run0190/taxonomy.qza --o-merged-data /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//taxonomy.qza 
Saved FeatureData[Taxonomy] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//taxonomy.qza 
Output file: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//taxonomy.qza 


# Alignment

## Creating alignment

In [13]:
aln_file = file.path(work_dir, 'aligned-rep-seqs_filt.qza')

cmd = 'qiime alignment mafft --p-n-threads {threads} --i-sequences {in_seq} --o-alignment {out_aln}'
cmd = glue::glue(cmd, threads=threads, in_seq=seqs_merged_file, out_aln=aln_file)
bash_job(cmd, conda_env=conda_env, stderr=TRUE)


Saved FeatureData[AlignedSequence] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//aligned-rep-seqs_filt.qza

NULL

## Masking alignment

In [14]:
aln_mask_file = file.path(work_dir, 'aligned-rep-seqs_filt_masked.qza')

cmd = 'qiime alignment mask --i-alignment {in_aln} --o-masked-alignment {out_aln}'
cmd = glue::glue(cmd, in_aln=aln_file, out_aln=aln_mask_file)
bash_job(cmd, conda_env=conda_env, stderr=TRUE)


Saved FeatureData[AlignedSequence] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//aligned-rep-seqs_filt_masked.qza

NULL

# Phylogeny

## Unrooted tree

In [15]:
phy_unroot_file = file.path(work_dir, 'aligned-rep-seqs_filt_masked_unroot-tree.qza')

cmd = 'qiime phylogeny fasttree --p-n-threads {threads} --i-alignment {in_aln} --o-tree {out_phy}'
cmd = glue::glue(cmd, threads=threads, in_aln=aln_mask_file, out_phy=phy_unroot_file)
bash_job(cmd, conda_env=conda_env, stderr=TRUE)


Saved Phylogeny[Unrooted] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//aligned-rep-seqs_filt_masked_unroot-tree.qza

NULL

## Rooted tree

In [16]:
phy_root_file = file.path(work_dir, 'aligned-rep-seqs_filt_masked_midroot-tree.qza')

cmd = 'qiime phylogeny midpoint-root --i-tree {in_phy} --o-rooted-tree {out_phy}'
cmd = glue::glue(cmd, in_phy=phy_unroot_file, out_phy=phy_root_file)
bash_job(cmd, conda_env=conda_env, stderr=TRUE)


Saved Phylogeny[Rooted] to: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged//aligned-rep-seqs_filt_masked_midroot-tree.qza

NULL

# sessionInfo

In [17]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.1 ggplot2_3.2.1     tidyr_1.0.0       dplyr_0.8.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       magrittr_1.5     munsell_0.5.0    tidyselect_0.2.5
 [5] uuid_0.1-2       colorspace_1.4-1 R6_2.4.1         rlang_0.4.2     
 [9] tools_3.6.2